In [ ]:
from collections import deque
from itertools import product

from math import inf
import numpy as np
import scipy.special
from sklearn.datasets import make_regression


def bfgs(f, f_grad, x0, max_iter=500, tol=1e-6):
    """
    BFGS algorithm for unconstrained optimization.

    Args:
        f: The objective function to be minimized.
        f_grad: The gradient of the objective function.
        x0: The initial guess for the solution.
        max_iter: The maximum number of iterations.
        tol: The tolerance for convergence.

    Returns:
        A tuple containing:
            - The optimal solution found.
            - The value of the objective function at the solution.
            - The number of iterations performed.
    """

    x = x0
    n = len(x0)
    B = np.eye(n)  # Initial approximation of the Hessian inverse
    grad = f_grad(x)

    for i in range(max_iter):
        # Compute search direction
        p = -np.linalg.solve(B, grad)

        # Perform line search
        t = line_search(f, f_grad, x, p)

        # Update x
        x_next = x + t * p

        # Update B (BFGS update)
        s = x_next - x
        y = f_grad(x_next) - grad
        if np.dot(s, y) > 0:  # Ensure positive definiteness
            rho = 1 / np.dot(s, y)
            B = (np.eye(n) - rho * np.outer(s, y)) @ B @ (np.eye(n) - rho * np.outer(y, s)) + rho * np.outer(s, s)
        else:
            B = np.eye(n)

        # Update x and gradient
        x = x_next
        grad = f_grad(x)

        # Check for convergence
        if np.linalg.norm(grad) < tol:
            break

    return x, f(x), i + 1


def line_search(f, f_grad, x, p, c1=1e-4, c2=0.9):
    """
    Backtracking line search.

    Args:
        f: The objective function.
        f_grad: The gradient of the objective function.
        x: The current point.
        p: The search direction.
        c1: The Armijo condition parameter.
        c2: The curvature condition parameter.

    Returns:
        The step size that satisfies the Wolfe conditions.
    """
    assert 0 < c1 < c2 < 1
    beta = inf
    alpha = 0.0
    t = 1.0
    s = np.dot(f_grad(x), p)

    for iter in range(10_000):
        # Check Armijo Condition
        if f(x + t * p) - f(x) <= c1 * t * s:
            # Check Wolfe Condition
            if f_grad(x + t * p) @ p >= c2 * s:
                break
            else:
                # Wolfe failed
                alpha = t
        else:
            # Armijo failed
            beta = t
        if beta < inf:
            t = (alpha + beta) / 2
        else:
            t = 2 * alpha

    if iter == 10_000 - 1:
        raise ValueError("Line search did not converge at x={x}, p={p}, t={t:.2e}".format(x=x, p=p, t=t))
    return t


# Example usage:
def f(x):
    return x[0] ** 2 + 5 * x[1] ** 4 + 0.1 * abs(x[0])


def f_grad(x, dx=1e-6):
    n = len(x)
    grad = np.zeros(n)
    f_val = f(x)
    for i in range(n):
        x_plus = x.copy()
        x_plus[i] += dx
        grad[i] = (f(x_plus) - f_val) / dx
    return grad


x0 = np.array([3.0, 4.0])
x_opt, f_opt, iterations = bfgs(f, f_grad, x0)

print("Optimal solution:", x_opt)
print("Optimal value:", f_opt)
print("Iterations:", iterations)

In [ ]:
from scipy.optimize import minimize, minimize_scalar
from sklearn.datasets import make_regression

x, y = make_regression(n_samples=100, n_features=10, noise=10, n_informative=3, random_state=42)


def f(beta):
    # enet criterion
    y_hat = x @ beta
    mse = np.mean((y - y_hat) ** 2)
    penalty = 10.0 * np.sum(np.abs(beta)) + 5 * np.sum(beta ** 2)
    return mse + penalty


beta_0 = np.zeros(shape=(x.shape[1],))
result = minimize(f, beta_0, method='BFGS')
print(result)


In [ ]:
# GD with line search Enet
from scipy.optimize import approx_fprime, minimize_scalar, minimize
from collections import deque


def f_enet(beta, l1: float, l2: float) -> float:
    # enet criterion
    y_hat = x @ beta
    mse = np.mean((y - y_hat) ** 2)
    penalty = l1 * np.sum(np.abs(beta)) + l2 * np.sum(np.square(beta))
    return mse + penalty


def f_grad_enet(beta, l1: float, l2: float) -> np.ndarray:
    grad_mse = x.T @ (x @ beta - y)
    grad_penalty = l1 * np.sign(beta) + l2 * beta
    return (grad_mse + grad_penalty) / x.shape[0]


def gd_with_line_search(l1: float, l2: float, max_iter=1_000, loss_rtol: float = 1e-8) -> np.ndarray:
    n, p = x.shape
    beta = np.zeros(p, dtype=np.float64)
    loss = f_enet(beta, l1, l2)
    g = np.zeros_like(beta)
    print_every = max_iter // 5
    grad_buffer = deque(maxlen=2)
    grad_buffer.append(np.zeros_like(beta))
    grad_buffer.append(np.zeros_like(beta))

    for i in range(max_iter):
        grad = f_grad_enet(beta, l1, l2)
        grad_buffer.append(grad)

        # Perform line search and convex combination of grads
        def h(x):
            t, alpha = x
            p = alpha * grad_buffer[0] + (1 - alpha) * grad_buffer[1]
            return f_enet(beta - t * p, l1, l2)

        res = minimize(h, [0.5, 0.5], bounds=[(1e-10, 1), (0, 1)])
        t, alpha = res.x
        p = alpha * grad_buffer[0] + (1 - alpha) * grad_buffer[1]
        new_beta = beta - t * p
        new_loss = f_enet(new_beta, l1, l2)

        if i % print_every == 0:
            print(
                f"Iteration {i:,}, loss {new_loss:.2e}, t {t:.2e}, alpha {alpha:.2f}, grad norm {np.linalg.norm(grad):.2e}")

        beta = new_beta
        loss = new_loss

    return beta


# [0.562, -0.0, 0.0, 5.217, -0.0, -0.0, 0.0, 0.368, 0.0, -0.0]
b = gd_with_line_search(10.0, 10.0, max_iter=100)
print([round(float(b_), 3) for b_ in b])

b_ridge = np.linalg.solve(x.T @ x + 1000 * np.eye(x.shape[1]), x.T @ y)
print([round(float(b_), 3) for b_ in b_ridge])


In [ ]:
def sgd_with_momentum(l1: float, l2: float, max_iter=1_000, mom_span=100, target_lr=1e-7) -> np.ndarray:
    n, p = x.shape
    beta = np.zeros(p, dtype=np.float64)
    loss = f_enet(beta, l1, l2)
    g = np.zeros_like(beta)
    t = 0.1
    decay = (target_lr / t) ** (1 / max_iter)
    print_every = max_iter // 5
    mom = 1 - 2 / mom_span

    for i in range(max_iter):
        # Sample a random data point
        idx = np.random.randint(0, n)
        x_i = x[idx]
        y_i = y[idx]

        # Calculate the gradient for the sampled data point
        grad = x_i * (x_i @ beta - y_i)  # Gradient of MSE for the data point
        grad_penalty = l1 * np.sign(beta) + l2 * beta  # Gradient of penalty
        grad += grad_penalty  # Add the penalty gradient

        g = (1 - mom) * grad + mom * g  # Update momentum
        p = -g  # Update direction

        new_beta = beta + t * p
        new_loss = f_enet(new_beta, l1, l2)

        if i % print_every == 0:
            print(f"Iteration {i:,}, loss {new_loss:.2e}, t {t:.2e}")

        beta = new_beta
        loss = new_loss
        t *= decay

    return beta


# [0.562, -0.0, 0.0, 5.217, -0.0, -0.0, 0.0, 0.368, 0.0, -0.0]
b = sgd_with_momentum(10, 10, max_iter=50_000, mom_span=300, target_lr=1e-8)
print([round(float(b_), 3) for b_ in b])


In [ ]:
from sklearn.linear_model import ElasticNet


def saga_enet(l1: float, l2: float, tol: float = 1e-6, max_iter: int = 50_000) -> np.ndarray:
    n, p = x.shape
    grad = np.zeros_like(x)
    g = np.zeros(x.shape[1])
    beta = np.zeros(x.shape[1])
    lr = 1e-1
    lr_target = 1e-10
    lr_factor = (lr_target / lr) ** (1 / max_iter)
    n_print = max_iter // 10

    for i in range(max_iter):
        idx = np.random.randint(n)
        prev = grad[idx, :]
        g -= prev
        x_i = x[idx, :]
        y_i = y[idx]
        grad_i = (
                     # MSE grad
                         (x_i @ beta - y_i) * x_i
                         # L1 grad
                         + l1 * np.sign(beta)
                         # L2 grad
                         + l2 * beta
                 ) / n
        grad[idx, :] = grad_i
        g += grad_i
        beta -= g * lr

        if np.linalg.norm(g) < tol:
            print(f"Converged at iteration {i:,}")
            break

        if i % n_print == 0:
            print(f"Iteration {i:,} grad norm: {np.linalg.norm(g):.2e}, lr: {lr:.2e}")
        lr *= lr_factor

    return beta


for l1, l2 in product([0.1, 1.0, 10.0], [0.1, 1.0, 10.0]):
    beta_saga = saga_enet(l1, l2)
    beta_cd = ElasticNet(alpha=l1 + l2, l1_ratio=l1 / (l1 + l2), fit_intercept=False).fit(x, y).coef_
    beta_gd = gd_with_line_search(l1, l2)

    print(f"\nl1={l1}, l2={l2}, diff={np.linalg.norm(beta_saga - beta_cd) / np.linalg.norm(beta_cd):.3%}")
    for b, name in zip([beta_saga, beta_cd, beta_gd], ["SAGA", "CD", "GD + LS"]):
        print(f"{name: >10}: {[round(float(b_), 3) for b_ in b]}")

# beta = saga_enet()
# print([round(float(b), 3) for b in beta])


In [175]:
def adam_enet(l1, l2, b1_span: int = 10, b2_span: int = 100, max_iter: int = 200_000) -> np.ndarray:
    beta_1 = 1 - 2 / b1_span
    beta_2 = 1 - 2 / b2_span
    n, p = x.shape
    beta = np.zeros(p)
    m = np.zeros(p)
    v = np.ones(p)
    epsilon = 1e-8
    alpha = 0.1
    target_lr = 1e-9
    decay = (target_lr / alpha) ** (1 / max_iter)
    print_every = max_iter // 3

    for t in range(1, max_iter + 1):
        i = np.random.randint(n)

        x_i = x[i]
        y_i = y[i]
        mse_grad = x_i * (x_i @ beta - y_i)
        penalty_grad = l1 * np.sign(beta) + l2 * beta
        grad = mse_grad + penalty_grad

        m = beta_1 * m + (1 - beta_1) * grad
        v = beta_2 * v + (1 - beta_2) * np.square(grad - m)

        p = -alpha * (m / (np.sqrt(v) + epsilon))
        beta += p

        if t % print_every == 0:
            loss = f_enet(beta, l1, l2)
            print(f"Iteration {t:,}, loss {loss:.2e}")

        alpha *= decay

    return beta


n, p = x.shape
# [0.562, -0.0, 0.0, 5.217, -0.0, -0.0, 0.0, 0.368, 0.0, -0.0]
b = adam_enet(10.0, 10.0, b1_span=10, b2_span=100, max_iter=80_000)
print([round(float(b_), 3) for b_ in b])

Iteration 26,666, loss 4.42e+03
Iteration 53,332, loss 4.42e+03
Iteration 79,998, loss 4.42e+03
[0.568, -0.0, -0.0, 5.052, -0.0, -0.0, 0.0, 0.308, 0.059, 0.0]


In [306]:
from sklearn.linear_model import SGDRegressor

l1 = 10.0
l2 = 10.0
reg = SGDRegressor(penalty='elasticnet', alpha=l1 + l2, l1_ratio=l1 / (l1 + l2), max_iter=10_000, tol=1e-6,
                   validation_fraction=0.0001, fit_intercept=False)
reg.fit(x, y)

print([round(float(b_), 3) for b_ in reg.coef_])

[0.82, 0.0, 0.0, 5.285, 0.0, 0.0, 0.0, 0.865, 0.0, 0.0]
